In [1]:
import numpy
import os
import glob
import matplotlib.pyplot as plt
from skimage import transform as tf
import math 
from skimage.transform import rescale
%matplotlib inline

import sys
sys.path.append('/vol/programs/keras-extras/utils/')

In [2]:
import keras

from keras.utils import np_utils

from keras.models import Model
from keras.layers import Input, merge, Convolution2D, Convolution3D, MaxPooling2D, UpSampling2D, RepeatVector, PReLU
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K

# from multi_gpu import *

Using TensorFlow backend.


In [3]:
# dataLocation = '/vol/data/numpyTraining/'
# trainingImages = 'padded_normalized_training_images.npy' #train_images.npy'
# trainingLabels = 'padded_normalized_training_labels.npy' #'train_labels.npy'

# img_rows = 320   
# img_cols = 256   
# slices = 100

img_rows = 128  
img_cols = 128   
slices = 64

smooth = 1.

In [4]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [5]:
def get_unet():
    inputs = Input((1, img_rows, img_cols))
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

    conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)
    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
    #learning rate (lr) was 1e-5

    return model 

modelTest = get_unet()
print(modelTest.summary())


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1, 128, 128)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 128, 128)  320         input_1[0][0]                    
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 128, 128)  9248        convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 64, 64)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [12]:
def get_unet():
    #input Data    
    inputs = Input(shape=(1, img_rows, img_cols, slices))
    #First convolusion, 16 new volumes. 
    conv1 = Convolution3D(16, 5, 5, 5, border_mode='same')(inputs)
    conv1PReLU = PReLU()(conv1)
    #Add the input to the result of the convolution (to do so, need to concatenate 16 versions of input)
    repeatedInput4 = merge([inputs, inputs, inputs, inputs], mode='concat', concat_axis=1)
    repeatedInput16 = merge([repeatedInput4, repeatedInput4, repeatedInput4,repeatedInput4], mode='concat', concat_axis=1)
    inputAdditionLayer = merge([repeatedInput16, conv1PReLU], mode='sum')
    
    ### FIRST DOWN CONVOLVE 
    #DownConvolve to get half the height/width/depth. Keep the volumes the same. 
    downConv1 = Convolution3D(16,2,2,2, border_mode='same', subsample=(2, 2,2))(inputAdditionLayer)
    downConv1PReLU = PReLU()(downConv1)
    
    ### Convolution Layer 2
    #Convolve the downconvolved twice. 5,5,5 convolutsions with 32 outputs. 
    conv2 = Convolution3D(32, 5, 5, 5, border_mode='same')(downConv1PReLU)
    conv2PReLU = PReLU()(conv2)
    conv2 = Convolution3D(32, 5, 5, 5, border_mode='same')(conv2PReLU)
    conv2PReLU = PReLU()(conv2)
    #Add the input theinput to convolution layer to the last convolution result. 
    repeatedDownConv1 = merge([downConv1PReLU, downConv1PReLU], mode='concat', concat_axis=1)
    conv2AdditionLayer = merge([repeatedDownConv1, conv2PReLU], mode='sum')
    
    ### SECOND DOWN CONVOLVE
    downConv2 = Convolution3D(32,2,2,2, border_mode = 'same', subsample=(2,2,2))(conv2AdditionLayer)
    downConv2PReLU = PReLU()(downConv2)
    
    ## Convolution Layer 3
    #convolve three times. 5,5,5, with 64 outputs. 
    conv3  = Convolution3D(64,5,5,5, border_mode='same')(downConv2PReLU)
    conv3PReLU = PReLU()(conv3)
    conv3  = Convolution3D(64,5,5,5, border_mode='same')(conv3PReLU)
    conv3PReLU = PReLU()(conv3)
    conv3  = Convolution3D(64,5,5,5, border_mode='same')(conv3PReLU)
    conv3PReLU = PReLU()(conv3)
    #Add the input theinput to convolution layer to the last convolution result. 
    repeatedDownConv2 = merge([downConv2PReLU, downConv2PReLU], mode='concat', concat_axis=1)
    conv3AdditionLayer = merge([repeatedDownConv2, conv3PReLU], mode='sum')
    
    ### THIRD DOWN CONVOLVE
    downConv3 = Convolution3D(64,2,2,2, border_mode = 'same', subsample=(2,2,2))(conv3AdditionLayer)
    downConv3PReLU = PReLU()(downConv3)
    
    ## Convolution Layer 4
    #convolve three times. 5,5,5, with 128 outputs. 
    conv4  = Convolution3D(128,5,5,5, border_mode='same')(downConv3PReLU)
    conv4PReLU = PReLU()(conv4)
    conv4  = Convolution3D(128,5,5,5, border_mode='same')(conv4PReLU)
    conv4PReLU = PReLU()(conv4)
    conv4  = Convolution3D(128,5,5,5, border_mode='same')(conv4PReLU)
    conv4PReLU = PReLU()(conv4)
    #Add the input theinput to convolution layer to the last convolution result. 
    repeatedDownConv3 = merge([downConv3PReLU, downConv3PReLU], mode='concat', concat_axis=1)
    conv4AdditionLayer = merge([repeatedDownConv3, conv4PReLU], mode='sum')
    
    ### FOURTH DOWN CONVOLVE
    downConv4 = Convolution3D(128,2,2,2, border_mode = 'same', subsample=(2,2,2))(conv4AdditionLayer)
    downConv4PReLU = PReLU()(downConv4)
    
    ## Convolution Layer 5
    #convolve three times. 5,5,5, with 256 outputs. 
    conv5  = Convolution3D(256,5,5,5, border_mode='same')(downConv4PReLU)
    conv5PReLU = PReLU()(conv5)
    conv5  = Convolution3D(256,5,5,5, border_mode='same')(conv5PReLU)
    conv5PReLU = PReLU()(conv5)
    conv5  = Convolution3D(256,5,5,5, border_mode='same')(conv5PReLU)
    conv5PReLU = PReLU()(conv5)
    #Add the input theinput to convolution layer to the last convolution result. 
    repeatedDownConv4 = merge([downConv4PReLU, downConv4PReLU], mode='concat', concat_axis=1)
    conv5AdditionLayer = merge([repeatedDownConv4, conv5PReLU], mode='sum')

    
    
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

#     conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
#     conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

#     conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
#     conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

#     conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
#     conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
#     pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

#     conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
#     conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

#     up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    
#     conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up6)
#     conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

#     up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
#     conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
#     conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

#     up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
#     conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
#     conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

#     up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
#     conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
#     conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

#     conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv5AdditionLayer)
    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
    #learning rate (lr) was 1e-5
    
    return model 

modelTest = get_unet()
print(modelTest.summary())


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 1, 128, 128, 6 0                                            
____________________________________________________________________________________________________
merge_49 (Merge)                 (None, 4, 128, 128, 6 0           input_8[0][0]                    
                                                                   input_8[0][0]                    
                                                                   input_8[0][0]                    
                                                                   input_8[0][0]                    
____________________________________________________________________________________________________
convolution3d_59 (Convolution3D) (None, 16, 128, 128,  2016        input_8[0][0]           

In [ ]:
# def preprocessLabels(labels):
#     labels[numpy.where(labels==1)] = 1
#     labels[numpy.where(labels==3)] = 1
#     labels[numpy.where(labels==2)] = 0
#     labels[numpy.where(labels==4)] = 0
#     return labels

In [ ]:
# def import_and_process(trainingImagesLocation, testingImagesLocation):
#     print('-'*30)
#     print('Loading and preprocessing train data...')
#     print('-'*30)
#     trainImages = (numpy.load(trainingImagesLocation)).astype(numpy.float32)
#     trainLabels = (numpy.load(testingImagesLocation)).astype(numpy.int8)
#     trainLabels = preprocessLabels(trainLabels)
#     return(trainLabels, trainImages)

In [ ]:
# trainLabels, trainImages = import_and_process(dataLocation + trainingImages, dataLocation + trainingLabels)

In [ ]:
# slice = 1050
# plt.figure(slice, figsize=(20,20))
# plt.subplot(211)
# plt.imshow(trainImages[slice,:,:], cmap ='gray')
# plt.subplot(212)
# plt.imshow(trainLabels[slice,:,:])

In [ ]:
# trainLabels = numpy.expand_dims(trainLabels, axis=1)
# trainImages = numpy.expand_dims(trainImages, axis=1)

In [ ]:
# print('Shape of Labels: ' + str(trainLabels.shape))
# print('Shape of Images: ' + str(trainImages.shape))

#### Parameters
Tried batch of 24 and it was running out of memory. 12 didnt run out of memory and was slightly faster.
batches of 12 lead to ~1hr/epoch processing time (just under & 3500+ s/epoch). 

Am trying batches of 16 to see if memory is still fine. Technically the higher the batch size will decrease training time, as long as still in memory. 
Also going to try and increase the learning rate of the optimizer. 
Scratch that... I reduced the pixel type to int 16 which should half the memory of the inputs. So, I should in theory be able to double the batch size. I have increased the batch size to 30 and will use that instead of 16. I have also increased the learning rate form 1e-5 to 1e-3.... that should also make convergence quicker... hopefully. 


18:28 Started to run after changing json keras file data type to float16. The file location was at ~/.keras/keras.json

The batch size = 24 
The smooth = 1 
The learning rate = 1e-6
epoch = 10 

10:46 Feb.1.2017: Changed the pre-processing so that all of the cartialge is background and the bones are 1's. This will create a network to segment the bones only. I think that this will be much "easier" and will likely create a great starting point. This might make it a 2 part segmentation. 
    - @ epoch1/10 and 5632/8715 the loss is 0.4834. I am stopping at this point to restart with mean centered and normalized data. I.e. subtract the mean and divide by the standard deviation. 
    - Another think to think about/try is just segmenting one bone per iteration. So, I would train a network to segment femur, a network to segment the tibia, etc. This could probably be done by first doing one bone, and then using the end weights of that network to segment the other bone, etc. 
    -17:08 - started training network with each image (whole 3d image) normalized. Normalization was done by subtracting the mean and dividing by the standard deviation. So, each image should have intensities between ~-4 and +4 where 95% of the contrasts are between -1.96 and +1.96.... Normalization was done by taking the mean and std assuming we ignored all zeros. This was done to remove potential bias for all of the background pixels that are == 0. 

In [ ]:
# def train(dataLocation):
#     print('-'*30)
#     print('Creating and compiling model...')
#     print('-'*30)
#     model = get_unet()
#     model_checkpoint = ModelCheckpoint(dataLocation + 'uNet_bones.hdf5', monitor='loss', save_best_only=True)
    
#     print('-'*30)
#     print('Fitting model...')
#     print('-'*30)
#     model.fit(trainImages, trainLabels, batch_size=16, nb_epoch=10, verbose=1, shuffle=True, validation_split=0.5,
#               callbacks=[model_checkpoint]) 
    
    

In [ ]:
# modelTest = get_unet()
# print(modelTest.summary())


In [ ]:
# train(dataLocation)